In [1]:
from keras.models import Model
import keras.models
import keras.layers as layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from keras import backend as K

import argparse
import feature_interpretation as cnna
import cnn_builder as cbuild
import cnn_runner as crun
import config
import csv
import dr_methods as drm
import niftiutils.helper_fxns as hf
import importlib
import itertools
from math import sqrt, log, pi, exp
import matplotlib.pyplot as plt
from numba import jit, njit, prange, vectorize, guvectorize
from numpy import matmul, diag
import numpy as np
import operator
import os
import pandas as pd
import random
import scipy
import time
import voi_methods as vm
%matplotlib inline

C:\Users\Clinton\AppData\Local\conda\conda\envs\old-keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
importlib.reload(vm)
C = config.Config()
T = config.Hyperparams()

In [ ]:
vm.reset_accnum('E100165687')

In [5]:
model = keras.models.load_model(os.path.join(C.model_dir, "prob_model_305.hdf5")) #models_305
model_dense_outputs = cbuild.build_pretrain_model(model)

In [7]:
orig_data_dict, num_samples = cbuild._collect_unaug_data()
filters_by_cls = {cls: model_dense_outputs.predict(orig_data_dict[cls][0], verbose=False) for cls in C.classes_to_include}
filter_results = np.concatenate([filters_by_cls[cls] for cls in C.classes_to_include], axis=0)

filter_avgs = np.mean(filter_results, axis=0)
filter_stds = np.std(filter_results, axis=0)

In [26]:
Z_test = ['E106097391_0', 'E104978772_1', '12900535_0', 'E100150242_0', 'E105490014_0', 'E103147618_0', 'E103510187_0', 'E104657225_0', 'E100551966_0', 'E101388602_0', 'E100215900_8', 'E100215900_7', 'E104045692_0', '13104521_0', 'E100383453_0', '12943286_0', '12271995_0', 'E102315724_0', 'E104949189_0', 'E100511083_1', 'E101579471_0', '13018986_1', '13203550_8', '13112385_0', '12712463_0', '12361082_0', '13028374_0', 'E103985934_1', 'E100529980_0', '12042703_3', '12961059_0', 'E105724706_2', 'E100592424_2', 'E103104254_0', 'E104546069_0', 'E101665217_1', '12090000_0', 'E100592424_1', '12961059_1', 'E105474285_0', '12502068_1', 'E100814791_0', 'E102613189_0', 'E105427046_0', 'E102881031_1', 'E102929168_0', 'E102310482_0', 'E102095465_0', 'E101811299_0', 'E104737273_0', '12890053_0', 'E100168661_1', '12637865_0', 'E100168661_2', '12239783_0', '12707781_0', '12706568_1', '12823036_0', '12404081_0', '12365693_1']
num_units = 100 # number of dense units
indices = np.array(hf.flatten([orig_data_dict[cls][1] for cls in C.classes_to_include]))
test_indices = np.where(np.isin(indices, Z_test))[0]
test_names = indices[test_indices]
imgs = np.array(hf.flatten([orig_data_dict[cls][0] for cls in C.classes_to_include]))
test_imgs = imgs[test_indices]

test_size=len(test_names)
voi_df = drm.get_voi_dfs()[0]

In [44]:
model_final_outputs = cbuild.build_pretrain_model(model, last_layer="pre-softmax")

In [ ]:
logits, uncertainties = mc_augment(test_names[test_id], model, voi_df)

logits = np.empty(aug_factor)
uncertainties = np.empty(aug_factor)
for aug_id in range(aug_factor):
    img = np.load(os.path.join(C.aug_dir, voi_row['cls'], "%s_%d.npy" % (test_names[test_id], aug_id)))
    logits[aug_id], uncertainties[aug_id] = model_dense_outputs.predict(np.expand_dims(img, 0))[0]

cls_accuracy[test_id] = sum(classif == C.classes_to_include.index(voi_row['cls']))/aug_factor
aug_means[test_id] = np.mean(outputs, 0)
aug_stds[test_id] = np.std(outputs, 0)

## Extra

### Variability in augmentation (epistemic)

In [ ]:
test_names[test_id]

In [48]:
aug_factor=100

aug_means = np.empty((test_size, num_units))
aug_stds = np.empty((test_size, num_units))
aug_accuracy = np.empty(test_size)

for test_id in range(test_size):
    voi_row = voi_df.loc[test_names[test_id]]
    
    outputs = np.empty((aug_factor, num_units))
    classif = np.empty(aug_factor)
    for aug_id in range(aug_factor):
        img = np.load(os.path.join(C.aug_dir, voi_row['cls'], "%s_%d.npy" % (test_names[test_id], aug_id)))
        outputs[aug_id] = model_dense_outputs.predict(np.expand_dims(img, 0))[0]
        classif[aug_id], _ = max(enumerate(model.predict(np.expand_dims(img, 0))[0]), key=operator.itemgetter(1))
    
    aug_accuracy[test_id] = sum(classif == C.classes_to_include.index(voi_row['cls']))/aug_factor
    aug_means[test_id] = np.mean(outputs, 0)
    aug_stds[test_id] = np.std(outputs, 0)

The standard deviation of the dense unit values is very large for the same lesion

### Variability in VOI selection (epistemic)

In [176]:
small_voi_df = pd.read_csv(C.small_voi_path)
dims_df = pd.read_csv(C.dims_df_path)

voi_means = np.empty((test_size, num_units))
voi_stds = np.empty((test_size, num_units))
voi_accuracy = np.empty(test_size)

shift_combinations = tuple([seq for seq in itertools.product([-5,0,5], repeat=6) if \
                            seq[1] >= seq[0] and seq[3] >= seq[2] and seq[5] >= seq[4]])
voi_factor=len(shift_combinations)

for test_id in range(test_size):
    lesion_id = test_names[test_id]
    coords = vm._get_voi_coords(small_voi_df[small_voi_df["id"] == lesion_id])
    voi_row = voi_df.loc[test_names[test_id]]
    
    outputs = np.empty((voi_factor, num_units))
    classif = np.empty(voi_factor)
    
    dims = dims_df[dims_df["AccNum"] == lesion_id[:lesion_id.rfind('_')]].iloc[0]
    shifts = [hf.flatten([x[:2]//dims['x'], x[2:4]//dims['y'], x[4:]//dims['z']]) for x in shift_combinations]
    for voi_id in range(voi_factor):
        dx = shifts[voi_id]#scipy.random.normal(0, 2, 3)
        #voi = list(map(int, hf.flatten([coords[:2]+dx[0], coords[2:4]+dx[1], coords[4:]+dx[2]])))
        voi = list(map(int, [coords[i]+dx[i] for i in range(6)]))
        img = vm.save_unaugmented_set(cls=voi_row["cls"], lesion_ids=[lesion_id], custom_vois=[voi],
                                      return_img_only=True)[0]
        img = np.expand_dims(img, 0)
        outputs[voi_id] = model_dense_outputs.predict(img)[0]
        classif[voi_id], _ = max(enumerate(model.predict(img)[0]), key=operator.itemgetter(1))
        
    voi_accuracy[test_id] = sum(classif == C.classes_to_include.index(voi_row['cls']))/voi_factor
    voi_means[test_id] = np.mean(outputs, 0)
    voi_stds[test_id] = np.std(outputs, 0)

### Variability with MC dropout (epistemic)

In [212]:
importlib.reload(cbuild)

<module 'cnn_builder' from 'C:\\Users\\Clinton\\Documents\\voi-classifier\\python\\cnn_builder.py'>

In [211]:
model_dropout = cbuild.build_model_dropout(model, 0.3)
mc_model_dense_outputs = cbuild.build_model_dropout(model, 0.3, "activation")

In [215]:
cnts = np.unique(classif, return_counts=True)

In [222]:
T = 50

mc_means = np.empty((test_size, num_units))
mc_stds = np.empty((test_size, num_units))
mc_certainty = np.empty(test_size)
mc_precision = np.empty(test_size)

for test_id in range(test_size):
    voi_row = voi_df.loc[test_names[test_id]]
    img = test_imgs[test_id]
    
    outputs = np.empty((T, num_units))
    classif = np.empty(T)
    for ix in range(T):
        outputs[ix] = mc_model_dense_outputs.predict(np.expand_dims(img, 0))[0]
        classif[ix], _ = max(enumerate(model_dropout.predict(np.expand_dims(img, 0))[0]), key=operator.itemgetter(1))
    
    cnts = np.unique(classif, return_counts=True)
    mc_precision[test_id] = C.classes_to_include[int(cnts[0][list(cnts[1]).index(max(cnts[1]))])] == voi_row['cls']
    mc_certainty[test_id] = sum(classif == C.classes_to_include.index(voi_row['cls']))/T
    mc_means[test_id] = np.mean(outputs, 0)
    mc_stds[test_id] = np.std(outputs, 0)

mc_accuracy = np.mean(mc_precision)